In [55]:
import pandas as pd
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

from collections import defaultdict
from gensim import corpora, models
from nltk.corpus import stopwords
from pprint import pprint


In [56]:
data = pd.read_csv("data_cleaned/combined.csv")
data.head()

,id,title,format,affiliation 1,affiliation 2,affiliation 3,affiliation 4,affiliation 5,affiliation 6,affiliation 7,...,Abstract,lower_abstract,affiliation,mode of presentation,affiliation 8,affiliation 9,affiliation 10,tags (primary),tags (secondary),affiliation 11
0,2013-1,Increasing the Effectiveness of Your Scholarly...,preconference workshop,University of Utah,Tecker International,NaN,NaN,NaN,NaN,NaN,...,This workshop is designed for those that are b...,this workshop is designed for those that are b...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-2,"Planning, Assessing, and Communicating Library...",preconference workshop,University of Illinois at Urbana-Champaign,Pierce College,NaN,NaN,NaN,NaN,NaN,...,Libraries in higher education are increasingly...,libraries in higher education are increasingly...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013-3,"Plugged into User Behavior: Low-Budget, High-I...",preconference workshop,University of Texas Arlington,NaN,NaN,NaN,NaN,NaN,NaN,...,How do students really use library subject gui...,how do students really use library subject gui...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013-4,Rediscover the Joy of Working Together!: Posit...,preconference workshop,Kansas State University,Rockhurst University,University of Kansas,University of Iowa,NaN,NaN,NaN,...,Join this collaborative session to enhance you...,join this collaborative session to enhance you...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013-5,Speculation to Litigation—Copyright and Climat...,preconference workshop,University of Louisville,Columbia University,Brigham Young University,Purdue University,NaN,NaN,NaN,...,Changes in digital technologies have produced ...,changes in digital technologies have produced ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
set(data["year"].tolist())

{2007, 2009, 2011, 2013, 2015, 2017, 2019, 2021}

In [62]:
documents = data["lower_abstract"].dropna().tolist()

In [77]:
stoplist = set(stopwords.words("english"))
stoplist.update(["library", "libraries"])
print(stoplist)

{'having', 'whom', 'hers', 'above', 'doesn', "don't", 'is', 'being', 'herself', 'no', "aren't", 'was', "it's", 't', 'by', 'needn', 'those', 'mustn', 'theirs', 'libraries', 'against', 'after', 'we', 'i', 'be', 'should', 'ain', 'few', 'because', 'more', 'here', 're', 'm', 's', "you've", 'wouldn', 'wasn', 'd', 'as', 'mightn', "should've", "couldn't", "haven't", 'haven', 'what', 'very', 'an', "that'll", 'own', 'he', "you're", "wouldn't", 'has', 'they', 'of', 'in', 'didn', 'himself', 'further', "doesn't", 'were', 'these', 'from', 'them', 'between', 'her', 'ours', 'with', 'over', 'such', 'not', 'don', 'a', 'same', 'o', 'hadn', "you'll", 'up', 'all', 'themselves', 'any', 'did', 'y', 'me', 'each', 'won', 'its', 'other', 'during', 'this', 'only', 'does', "you'd", 'most', 'll', "weren't", 'which', 'aren', 'off', "needn't", 'had', 'will', 'out', 'you', 'weren', 'who', 'how', 'why', 'at', 'some', 'their', 'she', 'have', 'when', "wasn't", 'yourself', "shan't", 'do', "shouldn't", 'isn', "mightn't", 

In [78]:
texts = [[word for word in document.lower().split() if word not in stoplist]
            for document in documents]

In [79]:
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

In [80]:
texts = [[token for token in text if frequency[token] > 1]
            for text in texts]

In [81]:
pprint(texts[:1])

[['workshop',
  'designed',
  'building',
  'programs',
  'institutions',
  'need',
  'strengthen',
  'revitalize',
  'local',
  'efforts.',
  'emphasis',
  'placed',
  'think',
  'strategically',
  'local',
  'programs',
  'develop',
  'skills',
  'influential',
  'without',
  'necessarily',
  'positional',
  'authority.',
  'workshop',
  'encourage',
  'participant',
  'interaction',
  'incorporating',
  'hands-on',
  'activities.']]


In [82]:
dictionary = corpora.Dictionary(texts)
dictionary.save('acrl.dict')

In [83]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('acrl.mm', corpus)

In [84]:
hdp = models.HdpModel(corpus, id2word=dictionary)

In [85]:
hdp.print_topics()

[(0,
  '0.008*librarians + 0.008*academic + 0.008*information + 0.007*students + 0.007*research + 0.005*learning + 0.005*new + 0.005*learn + 0.004*student + 0.004*literacy'),
 (1,
  '0.003*research + 0.003*librarians + 0.003*students + 0.002*information + 0.002*services + 0.002*faculty + 0.002*academic + 0.002*student + 0.001*learn + 0.001*online'),
 (2,
  '0.003*information + 0.003*learning + 0.002*literacy + 0.002*students + 0.002*learn + 0.002*librarians + 0.002*university + 0.002*student + 0.002*research + 0.002*academic'),
 (3,
  '0.002*librarians + 0.002*learning + 0.002*academic + 0.002*research + 0.002*information + 0.002*learn + 0.002*data + 0.002*student + 0.002*session + 0.001*open'),
 (4,
  '0.003*information + 0.002*research + 0.002*librarians + 0.002*literacy + 0.002*new + 0.002*students + 0.002*academic + 0.002*learn + 0.001*faculty + 0.001*use'),
 (5,
  '0.002*students + 0.002*librarians + 0.002*academic + 0.002*research + 0.001*information + 0.001*college + 0.001*liter

In [86]:
pyLDAvis.enable_notebook()

In [87]:
gensimvis.prepare(hdp, corpus, dictionary)

/Users/scott/projects/acrl_analysis/acrl/acrl_env/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.188354 -0.000072       1        1  74.079369
2      0.079925 -0.005228       2        1   2.267003
1      0.082828 -0.004346       3        1   2.160048
4      0.068310  0.000170       4        1   1.609030
3      0.064361 -0.010998       5        1   1.597841
...         ...       ...     ...      ...        ...
138   -0.014546  0.014451     146        1   0.004026
146   -0.014841 -0.026286     147        1   0.003934
140   -0.025114  0.002444     148        1   0.003557
144   -0.025436  0.018168     149        1   0.002876
149   -0.027789 -0.002552     150        1   0.000547

[150 rows x 5 columns], topic_info=             Term        Freq       Total  Category  logprob  loglift
75     librarians  631.000000  631.000000   Default  30.0000  30.0000
396      academic  626.000000  626.000000   Default  29.0000  29.0000
172   information  591.000000  591.000000   Default  28.0000  28.0000
83       students  557.000000  557.000000   Default  27.0000  27.0000
183      research  546.000000  546.000000   Default  26.0000  26.0000
...           ...         ...         ...       ...      ...      ...
1988     goodwill    0.000408   10.729975  Topic150  -7.2015   1.9395
7119    stressful    0.000400    9.164634  Topic150  -7.2207   2.0780
3087    designing    0.000426   22.760427  Topic150  -7.1593   1.2297
3705        suite    0.000404   11.324756  Topic150  -7.2127   1.8744
4532         hope    0.000403   14.822491  Topic150  -7.2152   1.6027

[8712 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
5614      1  0.286300     $60,000
489       1  0.896643           &
2716      1  0.440964         (1)
2716      3  0.146988         (1)
2716      4  0.146988         (1)
...     ...       ...         ...
7132      1  0.760927   ﬁrst-year
7022      1  0.423087    ﬁrsthand
7145      1  0.257106    ﬁve-year
7106      1  0.470473  ﬂexibility
7072      1  0.444109     ﬂexible

[6272 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2, 5, 4, 6, 7, 8, 10, 11, 12, 13, 17, 9, 18, 16, 19, 23, 26, 14, 21, 15, 22, 75, 20, 35, 32, 31, 49, 29, 83, 34, 40, 53, 44, 50, 24, 25, 30, 63, 36, 52, 62, 73, 42, 48, 51, 33, 61, 89, 39, 27, 67, 28, 100, 81, 43, 97, 60, 87, 58, 41, 55, 127, 54, 45, 101, 76, 71, 93, 65, 91, 95, 80, 92, 148, 138, 74, 37, 46, 66, 112, 133, 38, 143, 121, 88, 77, 94, 122, 79, 47, 96, 56, 59, 114, 137, 102, 98, 72, 142, 118, 136, 64, 84, 69, 107, 106, 70, 103, 134, 86, 99, 123, 57, 104, 68, 144, 85, 120, 129, 105, 140, 130, 146, 128, 149, 113, 124, 108, 115, 82, 116, 90, 132, 126, 110, 125, 135, 119, 78, 117, 131, 111, 109, 139, 147, 141, 145, 150])

In [88]:
lda = models.LdaModel(corpus, id2word=dictionary, num_topics=30)

In [89]:
lda.print_topics()

[(25,
  '0.009*"session" + 0.009*"beginners" + 0.006*"academic" + 0.006*"rights" + 0.006*"passing" + 0.006*"including" + 0.006*"learn" + 0.006*"work" + 0.005*"librarians" + 0.005*"collections"'),
 (18,
  '0.020*"learning" + 0.013*"librarians" + 0.012*"data" + 0.010*"research" + 0.009*"information" + 0.009*"learn" + 0.007*"literacy" + 0.007*"academic" + 0.007*"students" + 0.006*"campus"'),
 (8,
  '0.016*"copyright" + 0.013*"digital" + 0.009*"escape" + 0.007*"projects" + 0.007*"librarians" + 0.006*"information" + 0.006*"fair" + 0.006*"diversity," + 0.006*"work" + 0.006*"learning"'),
 (28,
  '0.057*"abstract" + 0.012*"research" + 0.010*"new" + 0.009*"academic" + 0.007*"university" + 0.007*"information" + 0.007*"students" + 0.005*"digital" + 0.005*"program" + 0.005*"services"'),
 (7,
  '0.020*"open" + 0.013*"new" + 0.009*"access" + 0.008*"librarians" + 0.007*"scholarly" + 0.007*"research" + 0.006*"work" + 0.006*"academic" + 0.006*"share" + 0.005*"learn"'),
 (13,
  '0.116*"abstract" + 0.008

In [91]:
gensimvis.prepare(lda, corpus, dictionary)

/Users/scott/projects/acrl_analysis/acrl/acrl_env/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
29     0.057105 -0.019201       1        1  7.448079
18     0.076720 -0.006916       2        1  7.435306
17     0.075752 -0.005425       3        1  6.476305
14     0.017312 -0.028329       4        1  5.466825
7     -0.041939 -0.031507       5        1  5.318284
1      0.063007 -0.032751       6        1  5.281892
5      0.028058 -0.014626       7        1  4.986731
4      0.051766  0.006102       8        1  4.560553
16     0.069286  0.009652       9        1  3.718612
12     0.022202 -0.000220      10        1  3.568334
11    -0.000332  0.013361      11        1  3.433368
10     0.059181 -0.003573      12        1  3.429736
0      0.034106 -0.015969      13        1  3.415612
26    -0.043510 -0.006070      14        1  2.884086
27     0.020561  0.006150      15        1  2.854542
9      0.060157  0.013718      16        1  2.716621
28    -0.054265 -0.141730      17        1  2.657579
22    -0.015682 -0.002028      18        1  2.414089
19     0.008540 -0.001603      19        1  2.402900
21    -0.039502  0.028418      20        1  2.320565
13    -0.084777 -0.148029      21        1  2.317387
15     0.052571  0.045515      22        1  2.104184
2     -0.063453  0.018913      23        1  1.995341
23    -0.021055 -0.001110      24        1  1.899643
6     -0.050553  0.029069      25        1  1.835146
25    -0.071506  0.038733      26        1  1.564561
8     -0.075873  0.082408      27        1  1.521946
24    -0.037350  0.043586      28        1  1.424814
3     -0.045173  0.057276      29        1  1.376577
20    -0.051356  0.066186      30        1  1.170381, topic_info=             Term        Freq       Total Category  logprob  loglift
5296     abstract  421.000000  421.000000  Default  30.0000  30.0000
174      literacy  484.000000  484.000000  Default  29.0000  29.0000
172   information  801.000000  801.000000  Default  28.0000  28.0000
138       digital  343.000000  343.000000  Default  27.0000  27.0000
751          open  235.000000  235.000000  Default  26.0000  26.0000
...           ...         ...         ...      ...      ...      ...
183      research    4.860402  855.745697  Topic30  -5.4851  -0.7230
174      literacy    4.646240  484.152417  Topic30  -5.5302  -0.1985
217     resources    4.508187  197.667958  Topic30  -5.5603   0.6671
649         tools    4.380171  177.851710  Topic30  -5.5892   0.7440
75     librarians    4.363746  893.104179  Topic30  -5.5929  -0.8735

[2201 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
2716      4  0.094061          (1)
2716     18  0.282182          (1)
2716     28  0.564364          (1)
2717      4  0.093884          (2)
2717     18  0.281652          (2)
...     ...       ...          ...
7460      8  0.729213            “
4507      1  0.851689           “a
4534     15  0.825040         “how
3548     28  0.590729       “maker
5124      5  0.746985  “publishing

[8566 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[30, 19, 18, 15, 8, 2, 6, 5, 17, 13, 12, 11, 1, 27, 28, 10, 29, 23, 20, 22, 14, 16, 3, 24, 7, 26, 9, 25, 4, 21])